# TLS handshake overview

In [ ]:
# We're going to parse several successive records from the passive listening of a standard TLS handshake
from scapy.all import *

## (C) ---> (S) ClientHello

In [ ]:
record1 = TLS(open('raw_data/tls_session_protected/01_cli.raw').read())
record1.show()

In [ ]:
for extension in record1.msg[0].ext:
    print ''
    extension.show()

## (C) <--- (S) ServerHello

In [ ]:
record2 = TLS(open('raw_data/tls_session_protected/02_srv.raw').read())
record2.show()

## (C) <--- (S) Certificate

In [ ]:
record3 = TLS(open('raw_data/tls_session_protected/03_srv.raw').read())
record3.show()

In [ ]:
for cert in record3.msg[0].certs:
    print ''
    cert[1].show()

In [ ]:
# Use the 'x509Cert' attribute to access the ASN.1 object underneath the crypto-enhanced wrapper
server_cert = record3.msg[0].certs[0][1]
print type(server_cert)
print type(server_cert.x509Cert)
print ''
server_cert.x509Cert.show()

## (C) <--- (S) CertificateStatus, ServerKeyExchange, ServerHelloDone

In [ ]:
# Here the server sent three TLS records in the same TCP segment
record4 = TLS(open('raw_data/tls_session_protected/04_srv.raw').read())
record4.show()

## (C) ---> (S) ClientKeyExchange, ChangeCipherSpec, Finished

In [ ]:
record5_str = open('raw_data/tls_session_protected/05_cli.raw').read()
record5 = TLS(record5_str)
record5.show()

In [ ]:
# Every record has a 'tls_session' context which may enhance the parsing of later records
record5 = TLS(record5_str, tls_session=record2.tls_session.mirror())
record5.show()

## (C) <--- (S) NewSessionTicket, ChangeCipherSpec, Finished

In [ ]:
record6_str = open('raw_data/tls_session_protected/06_srv.raw').read()
record6 = TLS(record6_str, tls_session=record5.tls_session.mirror())
record6.show()

## (C) ---> (S) ApplicationData

In [ ]:
record7_str = open('raw_data/tls_session_protected/07_cli.raw').read()
record7 = TLS(record7_str, tls_session=record6.tls_session.mirror())
record7.show()